# Mental Health NLP Model

## Load and explore data

In [1]:
import gensim.models
import numpy as np
import pandas as pd
import re
import gensim

df = pd.read_csv('data/winemag-data-130k-v2.csv')
df.head()


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [2]:
df.info()
df.points.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


88     17207
87     16933
90     15410
86     12600
89     12226
91     11359
92      9613
85      9530
93      6489
84      6480
94      3758
83      3025
82      1836
95      1535
81       692
96       523
80       397
97       229
98        77
99        33
100       19
Name: points, dtype: int64

In [3]:
# split points into binary label (80-89 = bad, 90-99 = good)
df['label'] = df['points'].apply(lambda x: 'good' if x > 89 else 'bad')

df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,label
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,bad
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,bad
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,bad
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,bad
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,bad


# Classification by Quality (Good / Bad)

## TF/IDF Approach

In [4]:
from sklearn.model_selection import train_test_split

X = df['description']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

pipe = Pipeline([('tfidf', TfidfVectorizer()), ('model', MultinomialNB())])
predictor = pipe.fit(X_train, y_train)

In [6]:
from sklearn import metrics

predictions = pipe.predict(X_test)

y_pred = pipe.predict(X_test)
print(metrics.classification_report(y_test, y_pred))
print(f'Accuracy: {metrics.accuracy_score(y_test, y_pred)}')

              precision    recall  f1-score   support

         bad       0.80      0.92      0.85     16201
        good       0.82      0.62      0.71      9794

    accuracy                           0.81     25995
   macro avg       0.81      0.77      0.78     25995
weighted avg       0.81      0.81      0.80     25995

Accuracy: 0.8061165608770917


The achieved Accuracy with the TF/IDF Vectorization is around 81% which is pretty good. But let's see if Word Embeddings can help to achieve a better score.

## Word Embedding Approach

### Self created Word Embedding

In [7]:
clean_txt = []

# TODO: find a better cleaning approach!
for w in range(len(df.description)):
    # make text lower case
    desc = df['description'][w].lower()

    #remove punctuation
    desc = re.sub('[^a-zA-Z]', ' ', desc)

    #remove tags
    desc = re.sub('&lt;/?.*?&gt;', ' &lt;&gt; ', desc)

    #remove digits and special chars
    desc = re.sub('(\\d|\\W)+', ' ', desc)
    clean_txt.append(desc)

df['clean_desc'] = clean_txt
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,label,clean_desc
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,bad,aromas include tropical fruit broom brimstone ...
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,bad,this is ripe and fruity a wine that is smooth ...
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,bad,tart and snappy the flavors of lime flesh and ...
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,bad,pineapple rind lemon pith and orange blossom s...
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,bad,much like the regular bottling from this comes...


In [8]:
corpus = []
for col in df.clean_desc:
    word_list = col.split(' ')
    corpus.append(word_list)

# generate vectors from corpus
modelMinCountOne = gensim.models.Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)
modelMinCountTwo = gensim.models.Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=2, workers=4)

## Evaluate Vocab Size diff based on min_count 1 or 2
print(f'With min_count=1 the Word Embedding has a vocabulary size of {len(modelMinCountOne.wv)} words.')
print(f'With min_count=2 the Word Embedding has a vocabulary size of {len(modelMinCountTwo.wv)} words.')


# let's go with min_count 1 for now. Since it has a 50% bigger vocabulary
modelMinCountOne.save('embeddings\description_emb.bin')
# modelMinCountTwo.save('embeddings\description_emb.model')

With min_count=1 the Word Embedding has a vocabulary size of 30463 words.
With min_count=2 the Word Embedding has a vocabulary size of 20243 words.


In [10]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB

# model = gensim.models.Word2Vec.load('embeddings\description_emb.model')
word_embedding = gensim.models.KeyedVectors.load('embeddings\description_emb.bin')

X = df['clean_desc']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def sentence_to_vector(sentence):
    vector = []
    for word in sentence.split():
        if word in word_embedding.wv:
            vector.append(word_embedding.wv[word])
    return np.mean(vector, axis=0)

# Convert your training data into feature vectors using your word embedding model
train_vectors = [sentence_to_vector(sentence) for sentence in X_train]
test_vectors = [sentence_to_vector(sentence) for sentence in X_test]

# Train your Naive Bayes model using your training data and labels
nb_model = BernoulliNB()
nb_model.fit(train_vectors, y_train)

BernoulliNB()

In [14]:
y_pred = nb_model.predict(test_vectors)

accuracy = accuracy_score(y_test, y_pred)
print(f'Word Embedding Accuracy: {accuracy}')

Word Embedding Accuracy: 0.6876322369686478


### Using the Google News Embedding

In [ ]:
# TODO: Continue that later https://machinelearningmastery.com/develop-word-embeddings-python-gensim
# Other Pre Trained Embeddings: https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

#from gensim.models import KeyedVectors

#filename = 'GoogleNews-vectors-negative300.bin'
#model = KeyedVectors.load_word2vec_format(filename, binary=True)

# Classification by Reviewers

## Data Preparation

In [ ]:
# find top 2 reviewers
df.taster_name.value_counts()

In [ ]:
filtered_df = df.query('taster_name == "Roger Voss" | taster_name == "Michael Schachner"')

filtered_df['taster_name']

## TF/IDF Approach